In [ ]:
# Let's build our project

# Here we are building a mulitple input mixed data model that takes in the 3 inputs seperately. RBI, chaneel matrices remain 
# seperate but now instead of giving Spectrum Occupancy matrix as input we give only NSA and instead of Src Des matrix we 
# just give Src and Des node id. The NSA and Src_Des input are given in two different branches, in the next model they will 
# be given together on one branch. The output power are indices rather matrices. So its an Nx1 vector.
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print(tf.config.list_physical_devices())

# number of iterations the matlab code ran or the number of times the network is simulated
itr = 24629
# Number of iterations the test data generation matlab code ran or the number of times the network is simulated
itr_test = 151
# number of nodes in the network
N = 25
# Number of nodes the imported model is trained on
N_import = 30
# We scale the non zero values of output power so that there is some sizable error if the model outputs a zero matrix
scaling_parameter = 1000

# Reading the RBI data
df = pd.read_csv("Data_24629_itr(N=25)/Data_rbi_vectors_25.csv")
#print(df)
df.shape

In [ ]:
# coverting it into numpy array
tem = df.to_numpy()
#print(tem)
tem.shape

#Just for the 1332 dataset as the last array will be nans
#tem = tem[:-1]

In [ ]:
a=list(range(0, itr))
X_1 = np.zeros((itr,N))
for i in a:
    X_1[i:] = tem[i:]
X_1.shape

In [ ]:
# We pad the RBI vectors with zeros so that the shape matches with the shape we trained the imported model on
a=list(range(0, itr))
X_1_new = np.zeros((itr,N_import))
for i in a:
    a1 = X_1[i]
    a1 = np.pad(a1, (0, (N_import-N)), 'constant')
    X_1_new[i,:] = a1
    #print(X_1_new[i])
X_1_new.shape

In [ ]:
# Now we assign X_1_new to X_1 as the model below takes the variable X_1

X_1 = np.zeros((itr,N_import))
X_1 = X_1_new
X_1.shape

In [ ]:
# Reading the channel data
df1 = pd.read_csv("Data_24629_itr(N=25)/Data_channel_matrices_25.csv")
#print(df1)
df1.shape

In [ ]:
# coverting it into numpy array
X_2 = df1.to_numpy()

In [ ]:
X_2 = X_2.reshape((itr,N,N))
#print(X_2)
X_2 = np.nan_to_num(X_2)
X_2.shape

In [ ]:
# We pad the channel matrices with zeros so that the shape matches with the shape we trained the imported model on

#np.pad(numpy_array, ((pad_top, pad_bottom), (pad_left, pad_right)), 'constant')

X_2_new = np.zeros((itr, N_import, N_import))
a=list(range(0, itr))
for i in a:
    a_temp = X_2[i,:,:]
    X_2_new[i, :, :] = np.pad(a_temp, ((0, (N_import - N)), (0, (N_import - N))), 'constant')
X_2_new.shape

In [ ]:
# Now we assign X_2_new to X_2 as the model below takes the variable X_2

X_2 = np.zeros((itr, N_import, N_import))
X_2 = X_2_new
X_2.shape

In [ ]:
# Reading the Source_Destination_info data
df2 = pd.read_csv("Data_24629_itr(N=25)/Data_Source_Destination_info_matrix(only Src Des)_25.csv")
#print(df2)
df2.shape

In [ ]:
# coverting it into numpy array
X_3 = df2.to_numpy()
#print(X_3)
X_3.shape

In [ ]:
X_3 = X_3.astype(np.float64)
print(X_3.dtype)
#print(X_3)

In [ ]:
# Reading the Spectrum_occupancy data, i.e, NSA
df3 = pd.read_csv("Data_24629_itr(N=25)/Data_spectrum_matrix_25.csv")
#print(df3)
df3.shape

In [ ]:
# coverting it into numpy array
X_4 = df3.to_numpy()
#print(X_4)
X_4.shape

# Its already in float64 so no need to convert

In [ ]:
# Reading the output power data
df4 = pd.read_csv("Data_24629_itr(N=25)/Data_output1_25.csv")
#print(df4)
df4.shape

In [ ]:
# coverting it into numpy array
X_5 = df4.to_numpy()
#print(X_5)
print(X_5.shape)
X_5 = np.nan_to_num(X_5)

In [ ]:
# This Cell converts each of the (1,30) output of y_data to (1,6)

a=list(range(0, itr))
max1 = 0
count = 0
maxlen = 0
max_path = np.zeros(30)
max_path_ind = 0
X_5_new = np.zeros((itr,6))
for i in a:
    temp = X_5[i]
    maxlen = len(temp[np.nonzero(temp)])
    if maxlen > 6:
        X_5_new[i,:] = temp[0:6]
        temp_non_zero = temp[np.nonzero(temp)]
        X_5_new[i,5] = temp_non_zero[-1]
        continue
    X_5_new[i,:] = temp[0:6]
    #print("original")
    #print(X_5[i])
X_5_new.shape

In [ ]:
y_data = X_5_new.astype(np.float64)
print(y_data.dtype)
#print(y_data_temp)
y_data.shape

# No need to unravel it as its already a vector

In [ ]:
X_1.dtype, X_2.dtype, X_3.dtype, X_4.dtype, y_data.dtype

In [ ]:
X_1.shape, X_2.shape, X_3.shape, X_4.shape, y_data.shape

In [ ]:
# scaling parameter for y_data so model gets greater penalty for error and also since output is sparse

y_data = y_data*scaling_parameter

In [ ]:
# converting y_data to tensor

y_data = tf.constant(y_data)
print(y_data.shape)
#y_data = tf.reshape(y_data, [itr,1,1,N*N])
#print(y_data.shape)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers.core import Activation
from tensorflow.python.keras.layers.core import Dense
from tensorflow.python.keras.layers import Dropout
from tensorflow.python.keras.layers import Flatten
from tensorflow.python.keras.layers import Input
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import concatenate
#from keras.optimizers import Adam

In [ ]:
## Transfer Learning

#Loading the pre-trained model trained on N=30
model = keras.models.load_model('saved_model/my_model1_(28-08-2023)')




In [ ]:
model.summary()

In [ ]:
# Now we read the test data to evaluate our model


# Reading the Test RBI data
df_test = pd.read_csv("Test_Data_151_itr(N=25)/Test_Data_rbi_vectors_25.csv")
#print(df_test)
df_test.shape

In [ ]:
# coverting it into numpy array
tem_test = df_test.to_numpy()
#print(tem_test)
tem_test.shape
X_1_test = tem_test
X_1_test.shape

In [ ]:
# We pad the RBI vectors with zeros so that the shape matches with the shape we trained the imported model on
a=list(range(0, itr_test))
X_1_new = np.zeros((itr_test, N_import))
for i in a:
    a1 = X_1_test[i]
    a1 = np.pad(a1, (0, (N_import-N)), 'constant')
    X_1_new[i,:] = a1
    #print(X_1_new[i])
X_1_new.shape

In [ ]:
# Now we assign X_1_new to X_1_test as the model below takes the variable X_1_test

X_1_test = np.zeros((itr_test, N_import))
X_1_test = X_1_new
X_1_test.shape

In [ ]:
# Reading the Test channel data
df1_test = pd.read_csv("Test_Data_151_itr(N=25)/Test_Data_channel_matrices_25.csv")
#print(df1_test)
df1_test.shape

In [ ]:
# coverting it into numpy array
X_2_test = df1_test.to_numpy()
X_2_test.shape

In [ ]:
X_2_test = X_2_test.reshape((itr_test,N,N))
#print(X_2_test)
X_2_test = np.nan_to_num(X_2_test)
#X_2_test

In [ ]:
# We pad the channel matrices with zeros so that the shape matches with the shape we trained the imported model on

#np.pad(numpy_array, ((pad_top, pad_bottom), (pad_left, pad_right)), 'constant')

X_2_new = np.zeros((itr_test, N_import, N_import))
a=list(range(0, itr_test))
for i in a:
    a_temp = X_2_test[i,:,:]
    X_2_new[i, :, :] = np.pad(a_temp, ((0, (N_import - N)), (0, (N_import - N))), 'constant')
X_2_new.shape

In [ ]:
# Now we assign X_2_new to X_2 as the model below takes the variable X_2

X_2_test = np.zeros((itr_test, N_import, N_import))
X_2_test = X_2_new
X_2_test.shape

In [ ]:
# Reading the Test Source_Destination_info data

df2_test = pd.read_csv("Test_Data_151_itr(N=25)/Test_Data_Source_Destination_info_matrix(only Src Des)_25.csv")
#print(df2_test)
df2_test.shape

In [ ]:
# coverting it into numpy array
X_3_test = df2_test.to_numpy()
#print(X_3_test)
X_3_test.shape

In [ ]:
X_3_test = X_3_test.astype(np.float64)
print(X_3_test.dtype)
#print(X_3_test)

In [ ]:
# Reading the Test Spectrum_occupancy data, i.e, NSA

df3_test = pd.read_csv("Test_Data_151_itr(N=25)/Test_Data_spectrum_matrix_25.csv")
#print(df3_test)
df3_test.shape

In [ ]:
# coverting it into numpy array
X_4_test = df3_test.to_numpy()
#print(X_4_test)
X_4_test.shape

In [ ]:
X_4_test = X_4_test.astype(np.float64)
print(X_4_test.dtype)
#print(X_4_test)

In [ ]:
# Reading the Test output power data

df4_test = pd.read_csv("Test_Data_151_itr(N=25)/Test_Data_output1_25.csv")
#print(df4_test)
df4_test.shape

In [ ]:
# coverting it into numpy array
X_5_test = df4_test.to_numpy()
#print(X_5_test)
print(X_5_test.shape)
X_5_test = np.nan_to_num(X_5_test)

In [ ]:
# This Cell converts each of the (1,30) y_data_test output to (1,6)

a=list(range(0, itr_test))
max1 = 0
count = 0
maxlen = 0
max_path = np.zeros(30)
max_path_ind = 0
X_5_test_new = np.zeros((itr_test,6))
for i in a:
    temp = X_5_test[i]
    maxlen = len(temp[np.nonzero(temp)])
    if maxlen > 6:
        X_5_test_new[i,:] = temp[0:6]
        temp_non_zero = temp[np.nonzero(temp)]
        X_5_test_new[i,5] = temp_non_zero[-1]
        continue
    X_5_test_new[i,:] = temp[0:6]
    #print("original")
    #print(X_5[i])
X_5_test_new.shape

In [ ]:
y_data_test = X_5_test_new.astype(np.float64)
print(y_data_test.dtype)
#print(y_data_test)
y_data_test.shape

In [ ]:
X_1_test.dtype, X_2_test.dtype, X_3_test.dtype, X_4_test.dtype, y_data_test.dtype

In [ ]:
X_1_test.shape, X_2_test.shape, X_3_test.shape, X_4_test.shape, y_data_test.shape

In [ ]:
X_data_test = [X_1_test, X_2_test, X_3_test, X_4_test]

In [ ]:
# converting y_data_test to tensor

y_data_test = tf.constant(y_data_test)
print(y_data_test.shape)
#y_data_test = tf.reshape(y_data_test, [itr_test,1,1,N*N])
#print(y_data_test.shape)
y_data_test

In [ ]:
# We scale the non zero values of Test output power so that there is some sizable error if the model outputs a zero matrix
#scaling_parameter = 100

y_data_test = y_data_test*scaling_parameter
y_data_test

In [ ]:
# Now we make predictions using our model
import time

t = time.perf_counter()
y_pred = model.predict(X_data_test)
elapsed = time.perf_counter() - t
print(elapsed)
print(y_pred.shape)
print("For 1 prediction")
print(elapsed/itr_test)

In [ ]:
# Evaluate the model on the test

t1 = time.perf_counter()
model.evaluate(X_data_test, y_data_test)
elapsed1 = time.perf_counter() - t1
print(elapsed1)

In [ ]:
x_axis = np.arange(1,(6)+1)
x_axis.shape
ind = 3
print(y_pred[ind]/scaling_parameter)
y_data_test[ind]/scaling_parameter

In [ ]:
a = list(range(0, itr_test))
#for i in a:
#    plt.figure()
#    plt.plot(x_axis,y_data_test[i]/scaling_parameter,'b',x_axis,y_pred[i]/scaling_parameter,'r')
#plt.show()

In [ ]:
model.save('saved_model/TL_without_finetuning_N=25_take1(30-08-2023)')
